In [ ]:
import os
import re

import matplotlib.pyplot as plt
import numpy as np
import cv2
from pathlib import Path
from datetime import datetime

from PIL import Image as PILImage
import matplotlib.patches as patches

from MirrorExtractor.simple_mirror_extractor import SimpleMirrorExtractor
from MirrorFeatureExtractor.mirror_feature_extractor import MirrorFeatureExtractor

In [ ]:
def mark_mirrors_on_img(img_path, points_list):
    img_bgr = cv2.imread(str(img_path))
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    ax.imshow(img_rgb)

    for points in points_list:
        # === Polygon na podstawie 4 współrzędnych ===
        polygon = patches.Polygon(
                points,
                closed=True,
                linewidth=2,
                edgecolor='red',
                facecolor='red',
                alpha=0.3  # przezroczystość wypełnienia
            )
        ax.add_patch(polygon)

    plt.tight_layout()
    plt.show()

In [ ]:
MY_COMP_OUTPUT_DIR_PATH = "/home/pgliwny/Praca/Computer_vision_for_MAGIC/data/data/images_for_analysis"
my_comp_root = Path("/home/pgliwny/Praca/Computer_vision_for_MAGIC/data/data/2025/10/")

BASE_DIR = "/home/pgliwny/Praca/Computer_vision_for_MAGIC/data"
ROOT = Path(BASE_DIR) / "data/images_for_analysis"
MIRROR_POINTS_JSON = Path(BASE_DIR) / "points_IRCam.json"

In [ ]:
IMG_PATH = "/home/pgliwny/Praca/Computer_vision_for_MAGIC/data/data/images_for_analysis/IRCamM1T20251005_075000M.jpg"

mirror_extractor = SimpleMirrorExtractor(str(MIRROR_POINTS_JSON))
mirror_feature_extractor = MirrorFeatureExtractor()

feature_names = ['lbp_mean', 'lbp_std', 'lbp_entropy',
                 'glcm_contrast', 'glcm_dissimilarity',
                 'glcm_homogeneity', 'glcm_energy', 'glcm_correlation']

feature_matrix = np.zeros((249, 8))  # wypełniasz w pętli po lustrach

img_bgr = cv2.imread(str(IMG_PATH))
img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

for mirror_id in range(0, 249):
    mirror_crop = mirror_extractor.extract_mirror_gray(img_gray, mirror_id=mirror_id)
    feat=mirror_feature_extractor.extract_texture_features(mirror_crop)
    feature_matrix[mirror_id] = list(feat.values())


In [ ]:
plt.hist(feature_matrix[:, 2])

In [ ]:
def get_outlier_mirrors(feature_array, sigma=3):
    med = np.median(feature_array)
    mad = np.median(np.abs(feature_array - med))
    z = (feature_array - med)/ (mad*1.4826) # Mnożnik 1.4826 przelicza MAD na jednostki porównywalne ze standardowym odchyleniem (dla rozkładu normalnego).
    return np.where(np.abs(z)>sigma)

In [ ]:
glcm_contrast = feature_matrix[:, 3]
get_outlier_mirrors(glcm_contrast)

In [ ]:
get_outlier_mirrors(feature_matrix[:, 0])

In [ ]:
get_outlier_mirrors(feature_matrix[:, 1])

In [ ]:
get_outlier_mirrors(feature_matrix[:, 2])

In [ ]:
get_outlier_mirrors(feature_matrix[:, 4])

In [ ]:
get_outlier_mirrors(feature_matrix[:, 5])

In [ ]:
p_list = []
p_list.append(mirror_extractor.get_point_coords(40))
p_list.append(mirror_extractor.get_point_coords(48))
p_list.append(mirror_extractor.get_point_coords(71))
p_list.append(mirror_extractor.get_point_coords(75))
p_list.append(mirror_extractor.get_point_coords(167))
p_list.append(mirror_extractor.get_point_coords(184))
p_list.append(mirror_extractor.get_point_coords(228))

In [ ]:
mark_mirrors_on_img(IMG_PATH, p_list)

In [ ]:


mirror_extractor = SimpleMirrorExtractor(str(MIRROR_POINTS_JSON))
mirror_feature_extractor = MirrorFeatureExtractor()

feature_names = ['lbp_mean', 'lbp_std', 'lbp_entropy',
                 'glcm_contrast', 'glcm_dissimilarity',
                 'glcm_homogeneity', 'glcm_energy', 'glcm_correlation']

feature_matrix = np.zeros((249, 8))  # wypełniasz w pętli po lustrach

for photo_path in sorted(ROOT.glob("*1005*.jpg")):
    print(photo_path)
    img_bgr = cv2.imread(str(photo_path))  # Path → str required by OpenCV
    if img_bgr is None:
        print(f"  Warning: could not read {photo_path}, skipping.")
        continue
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    mirror_crop = mirror_extractor.extract_mirror_gray(img_gray, mirror_id=150)



In [ ]:
plt.imshow(mirror_crop)

In [ ]:
mirror_crop = mirror_extractor.extract_mirror(img_rgb, mirror_id=248)

In [ ]:
mirror_feature_extractor.extract_texture_features(mirror_crop)

In [ ]:
def get_datetime(photo_path):
    pattern = r"T(\d{4})(\d{2})(\d{2})_(\d{2})(\d{2})(\d{2})"
    match = re.search(pattern, photo_path)
    if match:
        dt = datetime(*map(int, match.groups()))
        return dt

In [ ]:
get_datetime(str(photo_path).split("/")[-1])